# Genetic Algorithm Implementation in Python
Genetic Algorithm Implementation in Python — By Ahmed F. Gad<br>
LinkedIn: https://www.linkedin.com/pulse/genetic-algorithm-implementation-python-ahmed-gad/<br>
AI 研習社：https://ai.yanxishe.com/page/TextTranslation/1207

Source code : https://github.com/ahmedfgad/GeneticAlgorithmPython <br>
My clone: https://github.com/hcchengithub/GeneticAlgorithmPython


In [1]:
import peforth

reDef unknown


## Tutorial Example
The tutorial starts by presenting the equation that we are going to implement. The equation is shown below:

Y = w1x1 + w2x2 + w3x3 + w4x4 + w5x5 + w6x6

The equation has 6 inputs (x1 to x6) and 6 weights (w1 to w6) as shown and inputs values are (x1,x2,x3,x4,x5,x6)=(4,-2,7,5,11,1). We are looking to find the parameters (weights) that maximize such equation. The idea of maximizing such equation seems simple. The positive input is to be multiplied by the largest possible positive number and the negative number is to be multiplied by the smallest possible negative number. But the idea we are looking to implement is how to make GA do that its own in order to know that it is better to use positive weight with positive inputs and negative weights with negative inputs. Let us start implementing GA.

At first, let us create a list of the 6 inputs and a variable to hold the number of weights as follows:

In [2]:
equation_inputs = [4,-2,3.5,5,-11,-4.7]
num_weights = len(equation_inputs)

In [3]:
%f equation_inputs -->
%f num_weights -->

equation_inputs --> [4, -2, 3.5, 5, -11, -4.7] (<class 'list'>)
num_weights --> 6 (<class 'int'>)


The next step is to define the initial population. Based on the number of weights, each chromosome (solution or individual) in the population will definitely have 6 genes, one gene for each weight. But the question is how many solutions per the population? There is no fixed value for that and we can select the value that fits well with our problem. But we could leave it generic so that it can be changed in the code. Next, we create a variable that holds the number of solutions per population, another to hold the size of the population, and finally, a variable that holds the actual initial population:

In [4]:
import numpy

sol_per_pop = 8  # solutions per population 

# Defining the population size.
pop_size = (sol_per_pop,num_weights) # The population will have sol_per_pop chromosome where each chromosome has num_weights genes.

#Creating the initial population.
new_population = numpy.random.uniform(low=-4.0, high=4.0, size=pop_size)

After importing the numpy library, we are able to create the initial population randomly using the numpy.random.uniform function. According to the selected parameters, it will be of shape (8, 6). That is 8 chromosomes and each one has 6 genes, one for each weight. After running this code, the population is as follows:

In [5]:
%f new_population -->

new_population --> [[ 3.80033505  1.58383306  0.34441345 -3.97294971 -2.0545639   1.91696291]
 [-0.80156132  3.71515659  2.01858181  1.84546834 -0.99489869  1.85116002]
 [-2.76718889 -0.03102506  2.3979051   1.52923718 -3.51906376  1.30515785]
 [ 3.24358283  0.29445254 -2.01138083  3.00421072 -1.67251916 -0.64765578]
 [ 3.84349443  3.53271285 -1.4365973  -3.5003999  -3.11163715  3.61938618]
 [ 0.72379629 -2.8766597   1.65318438  1.61809023 -0.73048057 -3.30397912]
 [ 0.61009978  1.50944496  0.14987273 -0.48782377 -3.70712185  3.0943413 ]
 [-0.89330535 -0.38445307  3.42800661 -0.87809331  1.89958306 -3.40190074]] (<class 'numpy.ndarray'>)


Note that it is generated randomly and thus it will definitely change when get run again.

After preparing the population, next is to follow the flowchart in figure 1. Based on the fitness function, we are going to select the best individuals within the current population as parents for mating (挑選最佳個體作為交配的親代). Next is to apply the GA variants (crossover and mutation) 基因洗牌 與 突變 to produce the offspring of the next generation, creating the new population by appending both parents and offspring, and repeating such steps for a number of iterations/generations. The next code applies these steps:

In [6]:
import GA
num_generations = 5

num_parents_mating = 4
for generation in range(num_generations):
    # Measuring the fitness of each chromosome in the population.
    fitness = GA.cal_pop_fitness(equation_inputs, new_population)


    # Selecting the best parents in the population for mating.
    parents = GA.select_mating_pool(new_population, fitness, 
                                      num_parents_mating)

    # Generating next generation using crossover.
    offspring_crossover = GA.crossover(parents,
                                       offspring_size=(pop_size[0]-parents.shape[0], num_weights))

    # Adding some variations to the offsrping using mutation.
    offspring_mutation = GA.mutation(offspring_crossover)


    # Creating the new population based on the parents and offspring.
    new_population[0:parents.shape[0], :] = parents
    new_population[parents.shape[0]:, :] = offspring_mutation


The current number of generations is 5. It is selected to be small for presenting results of all generations within the tutorial. There is a module named GA that holds the implementation of the algorithm.

The first step is to find the fitness value of each solution within the population using the GA.cal_pop_fitness function. The implementation of such function inside the GA module is as follows:

In [9]:
def cal_pop_fitness(equation_inputs, pop):
    # Calculating the fitness value of each solution in the current population.
    # The fitness function calculates the sum of products between each input and its corresponding weight.
    fitness = numpy.sum(pop*equation_inputs, axis=1)
    return fitness

The fitness function accepts both the equation inputs values (x1 to x6) in addition to the population. The fitness value is calculated as the sum of product (SOP) between each input and its corresponding gene (weight) according to our function. According to the number of solutions per population, there will be a number of SOPs. As we previously set the number of solutions to 8 in the variable named sol_per_pop, there will be 8 SOPs as shown below:

In [18]:
SOPs = cal_pop_fitness(equation_inputs,new_population)
%f SOPs -->

SOPs --> [54.94894087 52.8509165  50.74982744 48.78357477 52.2820057  55.43250859
 44.12337941 56.44062476] (<class 'numpy.ndarray'>)


Note that the higher the fitness value the better the solution.

After calculating the fitness values for all solutions, next is to select the best of them as parents in the mating pool according to the next function GA.select_mating_pool. Such function accepts the population, the fitness values, and the number of parents needed. It returns the parents selected. Its implementation inside the GA module is as follows:

In [19]:
def select_mating_pool(pop, fitness, num_parents):
    # Selecting the best individuals in the current generation as parents for producing the offspring of the next generation.
    parents = numpy.empty((num_parents, pop.shape[1]))
    for parent_num in range(num_parents):
        max_fitness_idx = numpy.where(fitness == numpy.max(fitness))
        max_fitness_idx = max_fitness_idx[0][0]
        parents[parent_num, :] = pop[max_fitness_idx, :]
        fitness[max_fitness_idx] = -99999999999
    return parents

Based on the number of parents required as defined in the variable num_parents_mating, the function creates an empty array to hold them as in this line:

    parents = numpy.empty((num_parents, pop.shape[1]))

Looping through the current population, the function gets the index of the highest fitness value because it is the best solution to be selected according to this line:

    max_fitness_idx = numpy.where(fitness == numpy.max(fitness))
    
This index is used to retrieve the solution that corresponds to such fitness value using this line:

    parents[parent_num, :] = pop[max_fitness_idx, :]
    
To avoid selecting such solution again, its fitness value is set to a very small value that is likely to not be selected again which is -99999999999. The parents array is returned finally which will be as follows according to our example:    

In [24]:
parents_array = select_mating_pool(new_population, SOPs, num_parents_mating)
%f parents_array . cr  # 結果不太正常，哪裡出錯？

[[ 0.72379629 -2.8766597   1.65318438  3.00421072 -1.67251916 -1.5096865 ]
 [ 0.72379629 -2.8766597   1.65318438  3.00421072 -1.67251916 -1.5096865 ]
 [ 0.72379629 -2.8766597   1.65318438  3.00421072 -1.67251916 -1.5096865 ]
 [ 0.72379629 -2.8766597   1.65318438  3.00421072 -1.67251916 -1.5096865 ]]
結果不太正常，哪裡出錯？



Note that these three (四個吧？) parents are the best individuals within the current population based on their fitness values which are 18.24112489, 17.0688537, 15.99527402, and 14.40299221, respectively.

Next step is to use such selected parents for mating in order to generate the offspring. The mating starts with the crossover operation according to the GA.crossover function. This function accepts the parents and the offspring size. It uses the offspring size to know the number of offspring to produce from such parents. Such a function is implemented as follows inside the GA module:

In [25]:
def crossover(parents, offspring_size):
    offspring = numpy.empty(offspring_size)
    # The point at which crossover takes place between two parents. Usually, it is at the center.
    crossover_point = numpy.uint8(offspring_size[1]/2)

    for k in range(offspring_size[0]):
        # Index of the first parent to mate.
        parent1_idx = k%parents.shape[0]
        # Index of the second parent to mate.
        parent2_idx = (k+1)%parents.shape[0]
        # The new offspring will have its first half of its genes taken from the first parent.
        offspring[k, 0:crossover_point] = parents[parent1_idx, 0:crossover_point]
        # The new offspring will have its second half of its genes taken from the second parent.
        offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]
    return offspring

The function starts by creating an empty array based on the offspring size as in this line:

    offspring = numpy.empty(offspring_size)

Because we are using single point crossover, we need to specify the point at which crossover takes place. The point is selected to divide the solution into two equal halves according to this line:

    crossover_point = numpy.uint8(offspring_size[1]/2)

Then we need to select the two parents to crossover. The indices of these parents are selected according to these two lines:

    parent1_idx = k%parents.shape[0]
    parent2_idx = (k+1)%parents.shape[0]

The parents are selected in a way similar to a ring. The first with indices 0 and 1 are selected at first to produce two offspring. If there still remaining offspring to produce, then we select the parent 1 with parent 2 to produce another two offspring. If we are in need of more offspring, then we select the next two parents with indices 2 and 3. By index 3, we reached the last parent. If we need to produce more offspring, then we select parent with index 3 and go back to the parent with index 0, and so on.

The solutions after applying the crossover operation to the parents are stored into the offspring variable and they are as follows:

In [ ]:
offsprings = crossover(parents, offspring_size)
%f offsprings -->

Next is to apply the second GA variant, mutation, to the results of the crossover stored in the offspring variable using the mutation function inside the GA module. Such function accepts the crossover offspring and returns them after applying uniform mutation. That function is implemented as follows:

In [26]:
def mutation(offspring_crossover):
    # Mutation changes a single gene in each offspring randomly.
    for idx in range(offspring_crossover.shape[0]):
        # The random value to be added to the gene.
        random_value = numpy.random.uniform(-1.0, 1.0, 1)
        offspring_crossover[idx, 4] = offspring_crossover[idx, 4] + random_value
    return offspring_crossover

It loops through each offspring and adds a uniformly generated random number in the range from -1 to 1 according to this line:

    random_value = numpy.random.uniform(-1.0, 1.0, 1)

Such random number is then added to the gene with index 4 of the offspring according to this line:

    offspring_crossover[idx, 4] = offspring_crossover[idx, 4] + random_value
    
Note that the index could be changed to any other index. The offspring after applying mutation are as follows:

    [[-0.63698911 -2.8638447   2.93392615 -0.72163167  1.66083721  0.00677938]
     [ 3.00912373 -2.745417    3.27131287 -1.56909315 -1.94513681  2.29682254]
     [ 1.96561297  0.51030292  0.52852716  3.78571392  0.45337472  3.5170347 ]
     [ 2.12480298  2.97122243  3.60375452 -1.40103767 -1.5781162   0.30567304]]

In [ ]:
offspring = mutation(offspring_crossover)
%f offspring --> 

Such results are added to the variable offspring_crossover and got returned by the function.

At this point, we successfully produced 4 offspring from the 4 selected parents and we are ready to create the new population of the next generation.

Note that GA is a random-based optimization technique. It tries to enhance the current solutions by applying some random changes to them. Because such changes are random, we are not sure that they will produce better solutions. For such reason, it is preferred to keep the previous best solutions (parents) in the new population. In the worst case when all the new offspring are worse than such parents, we will continue using such parents. As a result, we guarantee that the new generation will at least preserve the previous good results and will not go worse. The new population will have its first 4 solutions from the previous parents. The last 4 solutions come from the offspring created after applying crossover and mutation:

    new_population[0:parents.shape[0], :] = parents
    new_population[parents.shape[0]:, :] = offspring_mutation

By calculating the fitness of all solutions (parents and offspring) of the first generation, their fitness is as follows:

    [ 18.24112489  17.0688537   15.99527402  14.40299221  -8.46075629  31.73289712   6.10307563  24.08733441]

The highest fitness previously was **18.24112489** but now it is **31.7328971158**. That means that the random changes moved towards a better solution. This is GREAT. But such results could be enhanced by going through more generations. Below are the results of each step for another 4 generations:

    Generation :  1

    Fitness values:

    [ 18.24112489  17.0688537   15.99527402  14.40299221  -8.46075629  31.73289712   6.10307563  24.08733441]

    Selected parents:

    [[ 3.00912373 -2.745417    3.27131287 -1.56909315 -1.94513681  2.29682254]
     [ 2.12480298  2.97122243  3.60375452 -1.40103767 -1.5781162   0.30567304]
     [-0.63698911 -2.8638447   2.93392615 -1.40103767 -1.20313655  0.30567304]
     [ 3.00912373 -2.745417    3.27131287 -0.72163167  0.7516408   0.00677938]]

    Crossover result:

    [[ 3.00912373 -2.745417    3.27131287 -1.40103767 -1.5781162   0.30567304]
     [ 2.12480298  2.97122243  3.60375452 -1.40103767 -1.20313655  0.30567304]
     [-0.63698911 -2.8638447   2.93392615 -0.72163167  0.7516408   0.00677938]
     [ 3.00912373 -2.745417    3.27131287 -1.56909315 -1.94513681  2.29682254]]

    Mutation result:

    [[ 3.00912373 -2.745417    3.27131287 -1.40103767 -1.2392086   0.30567304]
     [ 2.12480298  2.97122243  3.60375452 -1.40103767 -0.38610586  0.30567304]
     [-0.63698911 -2.8638447   2.93392615 -0.72163167  1.33639943  0.00677938]
     [ 3.00912373 -2.745417    3.27131287 -1.56909315 -1.13941727  2.29682254]]

    Best result after generation 1 :  34.1663669207
. . . snip . . .
    Generation :  4

    Fitness values

    [ 34.59304326  34.16636692  33.7449326   31.73289712  44.81692352

      33.35989464  36.46723397  37.19003273]

    Selected parents:

    [[ 3.00912373 -2.745417    3.27131287 -1.40103767 -2.20744102  0.30567304]
     [ 3.00912373 -2.745417    3.27131287 -1.56909315 -2.44124005  2.29682254]
     [ 3.00912373 -2.745417    3.27131287 -1.56909315 -2.37553107  2.29682254]
     [ 3.00912373 -2.745417    3.27131287 -1.56909315 -2.20515009  2.29682254]]

    Crossover result:

    [[ 3.00912373 -2.745417    3.27131287 -1.56909315 -2.37553107  2.29682254]
     [ 3.00912373 -2.745417    3.27131287 -1.56909315 -2.20515009  2.29682254]
     [ 3.00912373 -2.745417    3.27131287 -1.40103767 -2.20744102  0.30567304]]

    Mutation result:

    [[ 3.00912373 -2.745417    3.27131287 -1.56909315 -2.13382082  2.29682254]
     [ 3.00912373 -2.745417    3.27131287 -1.56909315 -2.98105233  2.29682254]
     [ 3.00912373 -2.745417    3.27131287 -1.56909315 -2.27638584  2.29682254]
     [ 3.00912373 -2.745417    3.27131287 -1.40103767 -1.70558545  0.30567304]]

    Best result after generation 4:  44.8169235189


After the above 5 generations, the best result now has a fitness value equal to 44.8169235189 compared to the best result after the first generation which is 18.24112489.

The best solution has the following weights:

    [3.00912373 -2.745417    3.27131287 -1.40103767 -2.20744102  0.30567304]


In [29]:
import numpy
import GA

"""

The y=target is to maximize this equation ASAP:

    y = w1x1 + w2x2 + w3x3 + w4x4 + w5x5 + 6wx6

    where (x1,x2,x3,x4,x5,x6)=(4,-2,3.5,5,-11,-4.7)

    What are the best values for the 6 weights w1 to w6?

    We are going to use the genetic algorithm for the best possible values after a number of generations.

"""

# Inputs of the equation.
equation_inputs = [4,-2,3.5,5,-11,-4.7]

# Number of the weights we are looking to optimize.
num_weights = 6



"""

Genetic algorithm parameters:

    Mating pool size

    Population size

"""

sol_per_pop = 8
num_parents_mating = 4

# Defining the population size.
pop_size = (sol_per_pop,num_weights) # The population will have sol_per_pop chromosome where each chromosome has num_weights genes.

#Creating the initial population.
new_population = numpy.random.uniform(low=-4.0, high=4.0, size=pop_size)

print(new_population)

num_generations = 500

for generation in range(num_generations):
    print("Generation : ", generation)

    # Measing the fitness of each chromosome in the population.
    fitness = GA.cal_pop_fitness(equation_inputs, new_population)

    # Selecting the best parents in the population for mating.
    parents = GA.select_mating_pool(new_population, fitness, num_parents_mating)

    # Generating next generation using crossover.
    offspring_crossover = GA.crossover(parents, offspring_size=(pop_size[0]-parents.shape[0], num_weights))

    # Adding some variations to the offsrping using mutation.
    offspring_mutation = GA.mutation(offspring_crossover)

    # Creating the new population based on the parents and offspring.
    new_population[0:parents.shape[0], :] = parents
    new_population[parents.shape[0]:, :] = offspring_mutation

    # The best result in the current iteration.
    print("Best result : ", numpy.max(numpy.sum(new_population*equation_inputs, axis=1)))



# Getting the best solution after iterating finishing all generations.
# At first, the fitness is calculated for each solution in the final generation.

fitness = GA.cal_pop_fitness(equation_inputs, new_population)

# Then return the index of that solution corresponding to the best fitness.

best_match_idx = numpy.where(fitness == numpy.max(fitness))



print("Best solution : ", new_population[best_match_idx, :])

print("Best solution fitness : ", fitness[best_match_idx])


[[ 0.97736244 -1.89217414  2.44826093  2.51852767 -3.13160072  1.45880228]
 [ 3.80878751 -1.55823379  3.90380604 -1.57275213  1.7579707   2.16069129]
 [ 2.00753923 -1.14707166  3.95754745 -2.81109284  1.16469628 -2.41577943]
 [ 3.44190304 -2.44232796  1.16267499 -1.91391314 -2.01119504 -3.06262173]
 [-1.91836871 -2.92100189  2.9979059   1.5649647  -1.268366    2.99392933]
 [ 1.9341553   1.51068093  0.96769676  0.01318784  1.31276318  0.00983388]
 [-2.10251268 -2.02988079 -3.24672583  3.01210802  1.9885925   1.71276166]
 [-2.26868151 -3.10270281 -1.5739872   0.00402387  1.63186924 -3.44289392]]
Generation :  0
Best result :  68.32872519129926
Generation :  1
Best result :  68.32872519129926
Generation :  2
Best result :  68.32872519129926
Generation :  3
Best result :  74.42907966537591
Generation :  4
Best result :  78.83394925206606
Generation :  5
Best result :  82.48378369939095
Generation :  6
Best result :  85.41497081198966
Generation :  7
Best result :  88.86910667919624
Generat

Generation :  233
Best result :  474.25576738616394
Generation :  234
Best result :  478.3434193124433
Generation :  235
Best result :  479.3159378584184
Generation :  236
Best result :  479.3934973910779
Generation :  237
Best result :  482.2982798960867
Generation :  238
Best result :  484.98668667151605
Generation :  239
Best result :  488.2043471857642
Generation :  240
Best result :  490.15689261398853
Generation :  241
Best result :  490.15689261398853
Generation :  242
Best result :  490.9739525596877
Generation :  243
Best result :  490.9739525596877
Generation :  244
Best result :  494.3066912520855
Generation :  245
Best result :  498.31866090397597
Generation :  246
Best result :  499.2486004328296
Generation :  247
Best result :  499.2486004328296
Generation :  248
Best result :  502.67952065906076
Generation :  249
Best result :  502.67952065906076
Generation :  250
Best result :  505.867912832946
Generation :  251
Best result :  506.0828220716582
Generation :  252
Best re